In [1]:
%matplotlib inline

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import sympy as sp
import pandas as pd

WARNING: I found many mistakes in this document on 04/22/17. It will need to be carefully reviewed again.

# Developing the constitutive equations for a single actuator

The development is based on my previous work, "Dynamic Modeling, Uncertainty, and Controller Design for Hydraulic Manipulator Trajectory Following".

## Pressure-Flow Dynamics

The flow through any orifice in the valve is given by the classical *orifice equation*:

$$Q = A C_d \sqrt{\frac{2}{\rho}|\Delta P_{ij}|} \ sgn(\Delta P_{ij}))$$

where $\Delta P_{ij}$ is the pressure differential across the orifice, $C_d$ is the empirical orifice discharge coefficient, $\rho$ is the fluid density, and A is the valve discharge area (i.e., a function of spool displacement, or in the case of the toy excavator, rotary valve angle).

Four such equations are necessary to characterize the four-port spool valves on each hydraulic actuator.

The relationship between spool position and orifice area, $A(s)$, is largely dependent on the type of valve, but here we will consider a open-center valve with

## Fluid Dynamics

The mass of fluid in the cap-end of the cylinder is described by:

$$m_A = \rho V_A$$

Further, if we consider fluid compressibility, the rate of change of mass is expressed by:

$$\dot{m}_A = \rho \dot{V}_A + \dot{\rho} V_A$$

### Fluid Compressibility in Cylinder

Compressibility is described by the fluid bulk modulus, $\beta$, and the following PDE:

$$\beta = \rho \left( \frac{\partial P}{\partial \rho} \right)$$

which at constant temperature, can be expressed as:

$$\beta \approx \rho \frac{\dot{P}}{\dot{\rho}}$$

## Flow Continuity

Compressibility is neglected for incoming fluid, giving the following:

$$\dot{m}_A = \rho Q_A$$

$$ Q_A = Q_{SA} - Q_{AR}$$

The rod position and velocity in the cylinder are given by:

$$V_A = A_{p,A}x + V_{A0}$$

$$\dot{V}_A = A_{p,A}\dot{x}$$

where $A_{p,A}$ is the cap-end pressurized area.

Synthesizing the above equations, we have:

$$\dot{P}_A = \frac{\beta}{A_{p,A}x+V_{A0}}(Q_{SA} - Q_{AR} - A_{p,A}\dot{x})$$

## Actuator Mechanics

For the purposes of our controller development we will consider inertial forces and digging forces on the actuator to be a lumped disturbance, $F_d$. Therefore,

$$\sum F = m\ddot{x} = P_A A_{p,A} - P_B A_{p,B} - F_d - b\dot{x}$$



___

# State Equations

Let us first define our states:

$x_1$ is rod position

$x_2 = \dot{x}_1$ (rod velocity)

$x_3 = P_{A}$ (cap-end (A) pressure) 

$x_4 = P_{B}$ (rod-end (B) pressure)

$$\dot{x}_1 = x_2$$

$$\dot{x}_2 = \frac{1}{m_{rod}+m_{load}}(x_3 A_{p,A}-x_4 A_{p,B} - F_L - bx_2)$$

$$\dot{x}_3 = \frac{\beta}{A_{p,A} x_1 + V_{A0}}(Q_{SA}-Q_{AR}-A_{p,A}x_2)$$

$$\dot{x}_4 = \frac{\beta}{V_{B0} + A_{p,B} (stroke - x_1)}(Q_{SB}-Q_{BR}+A_{p,B}x_2)$$

# Linearization

Let's repeat the above in Sympy, so that we can take the Jacobian.

In [ ]:
x1, x2, x3, x4 = sp.symbols('x1 x2 x3 x4')

# Jacobian

This is duplicated from my handwork on engr paper.

The Jacobian is formed as:

$$\frac{\partial F}{\partial x} = 
\begin{bmatrix}
J_{11} & J_{12} & J_{13} & J_{14} \\
J_{21} & J_{22} & J_{23} & J_{24} \\
J_{31} & J_{32} & J_{33} & J_{34} \\
J_{41} & J_{42} & J_{43} & J_{44}
\end{bmatrix}_{x_1, x_2, x_3, x_4}$$

where

$$J_{11} = 0$$

$$J_{12} = 1$$

$$J_{13} = 0$$

$$J_{14} = 0$$

$$J_{21} = 0$$

$$J_{22} = \frac{-b}{m_r}$$

$$J_{23} = \frac{A_{p,A}}{m_r}$$

$$J_{24} = \frac{A_{p,B}}{m_r}$$

$$J_{31} = -\beta A_{p,A} \left[ \frac{1}{A_{p,A} x_1 + V_{A0}} - \frac{Q_{SA} - Q_{AR} - A_{p,A} x_1}{(A_{p,A} x_1 + V_{A0})^2 } \right]$$


$$J_{32} = 0$$


$$J_{33} = \frac{\beta C_d}{A_{p,A} x_1 + V_{A0}} \left[ - \frac{A_{SA}}{\rho} \left( \frac{2}{\rho} \left| P_S - x_3 \right| \right)^{-\frac{1}{2}} - \frac{A_{AR}}{\rho} \left( \frac{2}{\rho} \left| x_3 - P_R \right| \right)^{-\frac{1}{2}} \right]$$


$$J_{34} = 0$$


$$J_{41} = -\beta A_{p,B} \left[ \frac{1}{V_{B0} - A_{p,B} x_1} - \frac{Q_{SB} - Q_{BR} + A_{p,B} x_1}{(V_{B0} - A_{p,B} x_1)^2} \right]$$


$$J_{42} = 0$$


$$J_{43} = 0$$


$$J_{44} = \frac{\beta C_d}{V_{B0} - A_{p,B} x_1} \left[ - \frac{A_{SB}}{\rho} \left( \frac{2}{\rho} \left| P_S - x_4 \right| \right)^{-\frac{1}{2}} - \frac{A_{AR}}{\rho} \left( \frac{2}{\rho} \left| x_4 - P_R \right| \right)^{-\frac{1}{2}} \right]$$